In [3]:
import pandas as pd

full = pd.read_csv('./results/Predictions_Full.csv')
lora = pd.read_csv('./results/Predictions_Lora.csv')


In [4]:
full.head()


,input_text,generated_text,reference_texts_0,reference_texts_1,reference_texts_2,predicted_label,reference_label
0,premise: This church choir sings to the masses...,The church choir can't sing joyous songs if th...,Not all churches have cracks in the ceiling,There is no indication that there are cracks i...,Not all churches have cracks in the ceiling.,contradiction,neutral
1,premise: This church choir sings to the masses...,A church choir sings joyous songs from the boo...,"""Filled with song"" is a rephrasing of the ""cho...",hearing song brings joyous in the church.,If the church choir sings then the church is f...,entailment,entailment
2,premise: This church choir sings to the masses...,A church choir is not a baseball game.,A choir sing some other songs other than book ...,The choir is at a chruch not a baseball game.,A baseball game isn’t played at a church.,contradiction,contradiction
3,"premise: A woman with a green headscarf, blue ...",Not all women are young.,the woman could've been old rather than young,There is no indication that the woman is young.,Not all women are young.,neutral,neutral
4,"premise: A woman with a green headscarf, blue ...",A woman with a big grin is not necessarily ver...,a grin suggests hapiness.,A WOMAN WITH BIG GRIN IS HAPPY,One must be happy in order to have a big grin.,entailment,entailment


# Label analysis

In [5]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

full_f1 = f1_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_f1 = f1_score(lora["reference_label"], lora["predicted_label"], average="weighted")
full_acc = accuracy_score(full["reference_label"], full["predicted_label"])
lora_acc = accuracy_score(lora["reference_label"], lora["predicted_label"])
full_prec = precision_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_prec = precision_score(lora["reference_label"], lora["predicted_label"], average="weighted")
full_rec = recall_score(full["reference_label"], full["predicted_label"], average="weighted")
lora_rec = recall_score(lora["reference_label"], lora["predicted_label"], average="weighted")

print(f"f1 lora={lora_f1} & f1 full={full_f1}")
print(f"acc lora={lora_acc} & acc full={full_acc}")
print(f"prec lora={lora_prec} & prec full={full_prec}")
print(f"recall lora={lora_rec} & recall full={full_rec}")


f1 lora=0.8832395589987994 & f1 full=0.9022181400980718
acc lora=0.8832451140065146 & acc full=0.902178338762215
prec lora=0.8832387590047901 & prec full=0.9023125710571344
recall lora=0.8832451140065146 & recall full=0.902178338762215


# Identify errors 

In [ ]:
# create csvs with wrong answers
df_full = full.assign(correct=full.predicted_label == full.reference_label)
full_false = df_full[df_full['correct'] == False].drop('correct', axis=1)

df_lora = lora.assign(correct=lora.predicted_label == lora.reference_label)
lora_false = df_lora[df_lora['correct'] == False].drop('correct', axis=1)

full_false.to_csv('./results/False_Predictions_Full.csv', index=True)
lora_false.to_csv('./results/False_Predictions_Lora.csv', index=True)

# find items that one model gets correct whereas the other doesn't
lora_false_list = lora_false.index.tolist()
full_false_list = full_false.index.tolist()

not_in_lora = [x for x in full_false_list if x not in lora_false_list]
not_in_full = [x for x in lora_false_list if x not in full_false_list]

print(not_in_lora)
print(len(not_in_lora))

print(not_in_full)
print(len(not_in_full))

